# AI Search with Vector Compression

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Load environment variables from .env file
load_dotenv(override=True)
%load_ext autoreload
%autoreload 2

In [2]:
# %pip install mercury

In [3]:
search_service_endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
search_service_key = os.environ["AZURE_SEARCH_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 3072))
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")

In [4]:
indexes = ["data-baseline"]

# read test_data.txt as array of strings
with open("test_data.txt", "r") as f:
    test_data = f.readlines()
    test_data = test_data[1:len(test_data)-1]
ds = pd.DataFrame(test_data, columns=["search_text"])
ds["search_text"] = ds["search_text"].str.strip()
ds.head(1)

,search_text
0,"""What is the main plot of the first Harry Pott..."


In [32]:
def search_docs(search_client, search_text):
    search_text = search_text.replace("\"", "")
    results = search_client.search(search_text=search_text, top=1)
    text = ""
    for res in results:
        text, score = res["chunk"], res["@search.score"]
    return text, score

search_client = SearchClient(endpoint=search_service_endpoint, index_name=indexes[0], credential=AzureKeyCredential(search_service_key))
ds["search_results"] = ds["search_text"].apply(lambda x: search_docs(search_client, x))
ds["search_results_text"] = ds["search_results"].apply(lambda x: x[0])
ds["search_results_score"] = ds["search_results"].apply(lambda x: x[1])
ds["search_results_score"] = ds["search_results_score"].astype(float)
# delete search_results column
ds = ds.drop(columns=["search_results"])
ds.head()

,search_text,search_results_text,search_results_score
0,"""What is the main plot of the first Harry Pott...","es of yellowish parchment, addressed in green ...",4.669311
1,"""Who are the main characters in Harry Potter a...","es of yellowish parchment, addressed in green ...",4.936015
2,"""What is the significance of the Sorting Hat i...","n their seats, squinting at the ground. “I ca...",5.508645
3,"""How does Harry Potter discover he is a wizard?"",",Harry Potter and the Chamber of Secrets by J...,4.458499
4,"""What are the key differences between the book...","es of yellowish parchment, addressed in green ...",6.495887
